# Proyecto ISIS-3302 (Modelado, Optimización y Simulación)

- Paulina Arrazola Vernaza - 202020631
- Santiago Alejandro Jaimes Puerto - 201912921
- Nicolás Rincón Sánchez - 202021963

# Etapa II (Implementación)

## 1. Desarrollo del Proyecto

### 1.1 Importación de Librerías

En el desarrollo de la implementación del proyecto, se requiere la instalación de librerías especializadas. En particular, se emplea como framework de optimización a **Pyomo** y se utilizan librerías para análisis de datos, como **Pandas**.

In [6]:
import pyomo.environ as pyo
import pandas as pd
import requests   # To make HTTP requests to OSRM API
import json       # To parse JSON responses
import time       # To potentially add delays for rate limiting
import sys        # For error messages
import math       # For fallback distance calc if needed

### 1.2 Configuración y Carga de datos

Para la extracción de datos de distancia por la calle (trayectos vehiculares sobre la malla vial de la ciudad) entre dos puntos A y B, se utilizó el API de OSRM. A continuación, se deja preestablecida la carga de los datos del caso 1.

In [7]:
# OSRM Server URL (Using the public demo server)
OSRM_BASE_URL = "http://router.project-osrm.org"

# Define the cost per kilometer (as per notebook)
C_KM = 20700 # COP/km

try:
    depots_df = pd.read_csv("case_2_base/Depots.csv")
    clients_df = pd.read_csv("case_2_base/Clients.csv")
    vehicles_df = pd.read_csv("case_2_base/Vehicles.csv")
    print("CSV files loaded successfully.")
except FileNotFoundError as e:
    print(f"ERROR: Could not find CSV file: {e}. Make sure the files are in the same directory.", file=sys.stderr)
    sys.exit(1) # Exit if data files are missing
except Exception as e:
    print(f"ERROR: An error occurred while loading CSV files: {e}", file=sys.stderr)
    sys.exit(1)

CSV files loaded successfully.


### 1.3 Preprocesamiento de Datos

Se realiza el procesamiento de los datos obtenidos desde el CSV para dejarlos en un formato propio de Python utilizable en futuros pasos.

In [8]:
# A. Pyomo-specific Sets
set_I_data = [f'D{depot_id}' for depot_id in depots_df['DepotID']]
set_J_data = [f'C{client_id}' for client_id in clients_df['ClientID']]
set_K_data = [f'V{i+1}' for i in range(len(vehicles_df))] # Vehicle IDs V1, V2, ...
set_N_data = set_I_data + set_J_data

# B. Pyomo-specific Parameters
param_A_data = {f'D{depots_df.loc[i, "DepotID"]}': float('inf') for i in depots_df.index}
print("WARNING: Depot capacities (A_i) not found in Depots.csv. Assuming infinite capacity.")

param_D_data = {f'C{clients_df.loc[i, "ClientID"]}': clients_df.loc[i, 'Product']
                for i in clients_df.index}
param_Q_data = {f'V{i+1}': vehicles_df.loc[i, 'Capacity'] for i in vehicles_df.index}
param_R_data = {f'V{i+1}': vehicles_df.loc[i, 'Range'] for i in vehicles_df.index}
param_n_cust_data = len(set_J_data)

# C. Data Dictionary for compute_distance_matrix function
data = {}
data['N'] = set_N_data # Set of Node IDs
data['UbicacionNodo'] = {} # Node coordinates {NodeID: (lat, lon)}

data['ID'] = list(vehicles_df['VehicleID'].unique()) # List of unique vehicle types

# Populate coordinates dictionary (lat, lon)
for i in depots_df.index:
    node_id = f'D{depots_df.loc[i, "DepotID"]}'
    data['UbicacionNodo'][node_id] = (depots_df.loc[i, 'Latitude'], depots_df.loc[i, 'Longitude'])
for i in clients_df.index:
    node_id = f'C{clients_df.loc[i, "ClientID"]}'
    data['UbicacionNodo'][node_id] = (clients_df.loc[i, 'Latitude'], clients_df.loc[i, 'Longitude'])

# Initialize the nested distance dictionary expected by the function
data['d'] = {tv: {u: {} for u in data['N']} for tv in data['ID']}

# D. Placeholder Parameters for Pyomo (will be filled by OSRM results)
param_d_data = {} # Will store {(u, v): distance_km}
param_c_data = {} # Will store {(u, v): cost_cop}

print("--- Data Prepared for Pyomo and Distance Function ---")
print(f"Set I (Depots): {set_I_data}")
print(f"Set J (Customers): {set_J_data}")
print(f"Set K (Vehicles): {set_K_data}")
print(f"Set N (Nodes): {data['N']}")
print(f"Param D (Demands): {param_D_data}")
print(f"Param Q (Veh. Caps): {param_Q_data}")
print(f"Param R (Veh. Ranges): {param_R_data}") 
print(f"Param n_cust: {param_n_cust_data}")
print(f"Vehicle ID (CV): {data['ID']}")
print(f"Node Locations (UbicacionNodo): {data['UbicacionNodo']}")
print("--- End Data Preparation ---")

--- Data Prepared for Pyomo and Distance Function ---
Set I (Depots): ['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12']
Set J (Customers): ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']
Set K (Vehicles): ['V1', 'V2', 'V3', 'V4', 'V5', 'V6']
Set N (Nodes): ['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']
Param D (Demands): {'C1': 12, 'C2': 15, 'C3': 15, 'C4': 6, 'C5': 5, 'C6': 11, 'C7': 12, 'C8': 10, 'C9': 15}
Param Q (Veh. Caps): {'V1': 131.9211396722696, 'V2': 108.4356199315333, 'V3': 91.50425520531196, 'V4': 32.896064077536955, 'V5': 22.65262807032524, 'V6': 22.682911535937688}
Param R (Veh. Ranges): {'V1': 145.85207096486445, 'V2': 1304.605971281605, 'V3': 953.172608610164, 'V4': 17.302304187458727, 'V5': 16.627680130757895, 'V6': 13.602810739289229}
Param n_cust: 9
Vehicle ID (CV): [1, 2, 3, 4, 5, 6]
Node Locations (UbicacionNodo): {'D1': (4.75021190869025, -74.0812421

### 1.4 Generación de la Matriz de distancias

En este paso, se calcula las distancias entre todos los nodos del grafo que incluye a los depósitos, los centros de distribución y los destinos. Recordemos que, como se mencionó al inicio, se utiliza el API de OSRM. Sin embargo, debido a que el API tiene una limitación respecto a la cantidad de nodos que puede procesar en una sola consulta, se tomó la decisión de dividir el conjunto de datos en pequeños batches.

Adicionalmente, se emplea un archivo de caché para evitar llamados redundantes al API.

In [9]:
import math, json, os, requests

OSRM_URL   = "https://router.project-osrm.org/table/v1/driving/"
CACHE_FILE = "osrm_cache_case2.json"     # se reutiliza entre ejecuciones
C_KM       = 20700                    # COP por km

# ------------------------------------------------------------------
# 1)  Construir lista de nodos y diccionario de coordenadas
#     (ya los tienes como set_N_data y data['UbicacionNodo'])
# ------------------------------------------------------------------
N      = data['N']
coords = data['UbicacionNodo']        # {node: (lat, lon)}

# ------------------------------------------------------------------
# 2)  Cargar caché (claves "u|v")
# ------------------------------------------------------------------
dist_cache = {}
if os.path.exists(CACHE_FILE):
    try:
        with open(CACHE_FILE) as f:
            raw = json.load(f)
        dist_cache = {tuple(k.split("|")): v for k, v in raw.items()}
        print(f"  Distancias cargadas de caché ({len(dist_cache)} pares).")
    except json.JSONDecodeError:
        print("  Caché dañada, se ignorará.")
        dist_cache = {}

# ------------------------------------------------------------------
# 3)  Identificar pares faltantes y, si hay, llamar a OSRM
# ------------------------------------------------------------------
missing = [(u, v) for u in N for v in N if (u, v) not in dist_cache]

if missing:
    print(f"Consultando OSRM para {len(missing)} pares…")
    MAX_COORDS = 100                                   # límite del servidor demo
    for start in range(0, len(N), MAX_COORDS):
        sub_nodes  = N[start:start+MAX_COORDS]
        coord_str  = ';'.join(f"{coords[n][1]},{coords[n][0]}" for n in sub_nodes)
        try:
            r = requests.get(OSRM_URL + coord_str,
                             params={"annotations": "distance"},
                             timeout=60)
            r.raise_for_status()
            matrix = r.json()["distances"]             # metros
        except Exception as e:
            print("  Error OSRM:", e, "→ se usa ∞")
            matrix = [[math.inf]*len(sub_nodes) for _ in sub_nodes]

        for i, u in enumerate(sub_nodes):
            for j, v in enumerate(sub_nodes):
                dist_cache[(u, v)] = matrix[i][j] / 1000.0   # a km

    # guardar caché
    with open(CACHE_FILE, "w") as f:
        json.dump({f"{u}|{v}": km for (u, v), km in dist_cache.items()}, f)
    print(" Caché OSRM actualizada.")
else:
    print("No faltaban distancias nuevas.")

# ------------------------------------------------------------------
# 4)  Rellenar param_d_data y param_c_data para Pyomo
# ------------------------------------------------------------------
param_d_data.clear()
param_c_data.clear()
for u in N:
    for v in N:
        km = dist_cache.get((u, v), math.inf)
        param_d_data[u, v] = km
        param_c_data[u, v] = km * C_KM
print(f"➜  Matriz de distancias y costos lista ({len(N)} × {len(N)} pares).")

# (opcional) mostrar un subconjunto
print("\nMuestra (primeros 4 nodos):")
for u in N[:4]:
    for v in N[:4]:
        print(f"d({u},{v})={param_d_data[u,v]:.2f} km, "
              f"c={param_c_data[u,v]:,.0f}", end="  |  ")
    print()


Consultando OSRM para 441 pares…


 Caché OSRM actualizada.
➜  Matriz de distancias y costos lista (21 × 21 pares).

Muestra (primeros 4 nodos):
d(D1,D1)=0.00 km, c=0  |  d(D1,D2)=33.06 km, c=684,367  |  d(D1,D3)=10.16 km, c=210,326  |  d(D1,D4)=6.45 km, c=133,575  |  
d(D2,D1)=32.95 km, c=681,997  |  d(D2,D2)=0.00 km, c=0  |  d(D2,D3)=34.18 km, c=707,530  |  d(D2,D4)=29.49 km, c=610,518  |  
d(D3,D1)=14.33 km, c=296,637  |  d(D3,D2)=38.91 km, c=805,480  |  d(D3,D3)=0.00 km, c=0  |  d(D3,D4)=15.35 km, c=317,679  |  
d(D4,D1)=5.16 km, c=106,723  |  d(D4,D2)=26.86 km, c=555,959  |  d(D4,D3)=10.57 km, c=218,780  |  d(D4,D4)=0.00 km, c=0  |  


### 1.5 Modelo Matematico en Pyomo

Se genera el modelo planteado por nosotros siguiendo los pasos de inicialización en Pyomo:
- Conjuntos
- Parámetros
- Variables de Decisión
- Restricciones

In [10]:
# Create a concrete model
model = pyo.ConcreteModel(name="LogistiCo_VRP_CSV_OSRM_UserFunc")

# --- Sets ---
model.I = pyo.Set(initialize=set_I_data, doc="Distribution Centers")
model.J = pyo.Set(initialize=set_J_data, doc="Customers")
model.K = pyo.Set(initialize=set_K_data, doc="Vehicles")
model.N = pyo.Set(initialize=set_N_data, doc="All Nodes")

# --- Parameters ---
model.A = pyo.Param(model.I, initialize=param_A_data, default=float('inf'), doc="Capacity of CD i")
model.D = pyo.Param(model.J, initialize=param_D_data, doc="Demand of customer j")
model.Q = pyo.Param(model.K, initialize=param_Q_data, doc="Capacity of vehicle k")
model.R = pyo.Param(model.K, initialize=param_R_data, doc="Range of vehicle k")

# Initialize with the extracted OSRM distances, default to infinity if missing
model.d = pyo.Param(model.N, model.N, initialize=param_d_data, default=float('inf'), doc="Distance from u to v (km)")
model.c = pyo.Param(model.N, model.N, initialize=param_c_data, default=float('inf'), doc="Cost from u to v (COP)")
model.n_cust = pyo.Param(initialize=param_n_cust_data, doc="Number of customers")

# --- Decision Variables ---
model.x = pyo.Var(model.N, model.N, model.K, within=pyo.Binary, doc="Vehicle k travels from u to v")
model.y = pyo.Var(model.I, model.K, within=pyo.Binary, doc="Vehicle k assigned to CD i")
model.u = pyo.Var(model.J, model.K, within=pyo.NonNegativeReals, bounds=(0, model.n_cust), doc="MTZ auxiliary variable") # Added bounds for clarity

# --- Objective Function ---
def objective_rule(mod):
    # Ensure cost is not infinite when calculating objective
    cost = sum(mod.c[u, v] * mod.x[u, v, k]
               for k in mod.K for u in mod.N for v in mod.N
               if u != v and mod.c[u, v] != float('inf'))
    # Add penalty for using arcs with infinite cost (should not happen if feasible)
    penalty = sum(1e9 * mod.x[u, v, k]
                   for k in mod.K for u in mod.N for v in mod.N
                   if u != v and mod.c[u, v] == float('inf'))
    return cost + penalty
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize, doc="Minimize total cost")

# --- Constraints ---
# Constraint 1: Customer Coverage
def customer_coverage_rule(mod, j):
    return sum(mod.x[u, j, k] for k in mod.K for u in mod.N if u != j) == 1
model.customer_coverage = pyo.Constraint(model.J, rule=customer_coverage_rule, doc="Each customer visited exactly once")

# Constraint 2a: Flow Conservation at Customer Nodes
def flow_conservation_customer_rule(mod, j, k):
    in_flow = sum(mod.x[u, j, k] for u in mod.N if u != j)
    out_flow = sum(mod.x[j, v, k] for v in mod.N if v != j)
    return in_flow == out_flow
model.flow_conservation_customer = pyo.Constraint(model.J, model.K, rule=flow_conservation_customer_rule, doc="Flow conservation at customer nodes")

# Constraint 2b: Vehicle Departure from Assigned Depot
def vehicle_departure_rule(mod, i, k):
    return sum(mod.x[i, v, k] for v in mod.N if v != i) == mod.y[i, k]
model.vehicle_departure = pyo.Constraint(model.I, model.K, rule=vehicle_departure_rule, doc="Vehicle leaves assigned depot")

# Constraint 2c: Vehicle Return to Assigned Depot
def vehicle_return_rule(mod, i, k):
    return sum(mod.x[u, i, k] for u in mod.N if u != i) == mod.y[i, k]
model.vehicle_return = pyo.Constraint(model.I, model.K, rule=vehicle_return_rule, doc="Vehicle returns to assigned depot")

# Constraint 2d: Vehicle Assignment Constraint
def vehicle_assignment_rule(mod, k):
    return sum(mod.y[i, k] for i in mod.I) <= 1
model.vehicle_assignment = pyo.Constraint(model.K, rule=vehicle_assignment_rule, doc="Each vehicle assigned to at most one depot")


# Constraint 3: Vehicle Capacity
def vehicle_capacity_rule(mod, k):
    demand_served_by_k = sum(mod.D[j] * sum(mod.x[u, j, k] for u in mod.N if u != j) for j in mod.J)
    return demand_served_by_k <= mod.Q[k]
model.vehicle_capacity = pyo.Constraint(model.K, rule=vehicle_capacity_rule, doc="Vehicle capacity constraint")

# Constraint 4: Vehicle Range (Autonomy)
def vehicle_range_rule(mod, k):
    # Ensure distance is not infinite when calculating range
    dist_traveled = sum(mod.d[u, v] * mod.x[u, v, k]
                       for u in mod.N for v in mod.N
                       if u != v and mod.d[u,v] != float('inf'))
    # Add a check to ensure no infeasible arcs are used
    infeasible_arc_used = sum(mod.x[u, v, k]
                              for u in mod.N for v in mod.N
                              if u != v and mod.d[u,v] == float('inf'))
    if pyo.value(infeasible_arc_used) > 0.1: # If solver uses an infinite arc
         return pyo.Constraint.Infeasible
    return dist_traveled <= mod.R[k]
model.vehicle_range = pyo.Constraint(model.K, rule=vehicle_range_rule, doc="Vehicle range constraint")

# Constraint 5: Distribution Center Capacity (Omitted)
print("Skipping CD Capacity Constraint (5).")

# Constraint 6: Subtour Elimination (MTZ)
def subtour_elimination_rule(mod, j, j_prime, k):
    if j == j_prime:
        return pyo.Constraint.Skip
    # Make sure arc exists before applying constraint
    if mod.d[j,j_prime] == float('inf'): # If no route exists between j and j', x should be 0
        return mod.x[j,j_prime,k] == 0
    return mod.u[j, k] - mod.u[j_prime, k] + mod.n_cust * mod.x[j, j_prime, k] <= mod.n_cust - 1
model.subtour_elimination = pyo.Constraint(model.J, model.J, model.K, rule=subtour_elimination_rule, doc="MTZ subtour elimination")

# Constraint 7: Prevent trivial loops (x_uuk = 0)
def no_trivial_loops_rule(mod, u, k):
    return mod.x[u, u, k] == 0
model.no_trivial_loops = pyo.Constraint(model.N, model.K, rule=no_trivial_loops_rule, doc="Prevent travel from a node to itself")

print("Pyomo model structure defined using data from CSVs and OSRM API (User Func).")
# --- End Python Code ---

Skipping CD Capacity Constraint (5).
Pyomo model structure defined using data from CSVs and OSRM API (User Func).


### 1.6 Ejecución del Solver

Se utiliza el solver `gurobi` incorporado dentro de la lógica de Pyomo para poder resolver el modelo y encontrar un óptimo, o por lo menos, una solución factible. 

In [14]:
# --- Markdown ---
# ## 4. Solve the Model and Display Results
#
# Use a selected MILP solver (e.g., GLPK, CBC) to find the optimal solution
# for the VRP model and display the results, including cost, assignments, and routes.
# --- End Markdown ---

# --- Python Code ---
solver_name = 'gurobi' # Or 'cbc'
try:
    solver = pyo.SolverFactory(solver_name)
    if not solver.available():
        raise RuntimeError(f"Solver '{solver_name}' not found or not executable.")
except Exception as e:
    print(f"ERROR: Could not find or initialize solver '{solver_name}'. Please install it and ensure it's in your PATH. Error: {e}", file=sys.stderr)
    print("You can try installing GLPK or CBC (e.g., using conda).", file=sys.stderr)
    sys.exit(1)


print(f"\nSolving the model using {solver_name}...")
# Add a time limit, e.g., 5 minutes, especially for larger problems
# results = solver.solve(model, tee=True, timelimit=300)
results = solver.solve(model, tee=True) # tee=True shows solver output

# --- Display Results ---

print("\n--- Solver Results ---")
print(results) # Print detailed solver results object

if results.solver.termination_condition == pyo.TerminationCondition.optimal or \
   (results.solver.termination_condition == pyo.TerminationCondition.feasible and len(results.solution) > 0) or \
   (results.solver.termination_condition == pyo.TerminationCondition.maxTimeLimit and len(results.solution) > 0):

    if results.solver.termination_condition == pyo.TerminationCondition.optimal:
        print("\n--- Optimal Solution Found ---")
    elif results.solver.termination_condition == pyo.TerminationCondition.maxTimeLimit:
         print("\n--- Time Limit Reached - Feasible Solution Found ---")
    else:
        print("\n--- Feasible Solution Found (may not be optimal) ---")

    print(f"Minimum Total Cost: {pyo.value(model.objective):,.2f} COP")

    print("\n--- Vehicle Assignments (y_ik) ---")
    assigned_vehicles_count = 0
    assignments = {}
    for k in model.K:
        assignments[k] = 'Unassigned'
        for i in model.I:
            # Use a tolerance for checking binary variable values
            if pyo.value(model.y[i, k], exception=False) is not None and pyo.value(model.y[i, k]) > 0.5:
                print(f"Vehicle {k} starts from CD {i}")
                assignments[k] = i
                assigned_vehicles_count += 1
                break
        if assignments[k] == 'Unassigned':
             print(f"Vehicle {k} is not used.")
    if assigned_vehicles_count == 0:
        print("No vehicles were assigned to any routes.")

    print("\n--- Routes (x_uvk) ---")
    total_distance = 0
    total_demand_served = 0
    vehicles_used = set()

    for k in model.K:
        start_depot = assignments.get(k)
        if start_depot != 'Unassigned':
            print(f"\nRoute for Vehicle {k} (from {start_depot}):")
            route = [start_depot]
            current_node = start_depot
            route_distance = 0
            route_demand = 0
            visited_nodes = {start_depot}
            route_found = False
            max_steps = len(model.N) + 2 # Increased safety margin

            for step in range(max_steps):
                next_node_found = False
                possible_next = []
                for v in model.N:
                    if current_node != v:
                         # Check value safely, handling potential None if variable wasn't solved
                         x_val = pyo.value(model.x[current_node, v, k], exception=False)
                         if x_val is not None and x_val > 0.5:
                            possible_next.append(v)
                            next_node_found = True

                if len(possible_next) > 1 and current_node != start_depot:
                     print(f"  ERROR: Multiple next steps found from {current_node} for vehicle {k}: {possible_next}")
                     route = [start_depot, "Error"] # Mark route as problematic
                     break
                elif not next_node_found and current_node != start_depot:
                     print(f"  ERROR: No next step found from {current_node} for vehicle {k}. Incomplete route.")
                     route.append("Error")
                     break
                elif not next_node_found and current_node == start_depot:
                    print(f"  Vehicle {k} assigned but seems to have no outgoing route from {start_depot}.")
                    route = [start_depot]
                    break

                if next_node_found:
                    next_node = possible_next[0]
                    arc_dist = model.d[current_node, next_node]
                    if arc_dist == float('inf'):
                        print(f"  ERROR: Route uses an infeasible arc from {current_node} to {next_node}")
                        route.append(f"{next_node}(inf)")
                        route_found = False # Mark as incomplete due to error
                        break
                    route_distance += arc_dist
                    route.append(next_node)
                    visited_nodes.add(next_node)

                    if next_node in model.J:
                        route_demand += model.D[next_node]

                    current_node = next_node

                    if current_node == start_depot:
                        route_found = True
                        break

                if step == max_steps - 1 and not route_found:
                    print(f"  ERROR: Route for vehicle {k} did not return to depot within {max_steps} steps.")
                    route.append("...") # Indicate incomplete


            # Print reconstructed route and details
            if route_found:
                 print(f"  {' -> '.join(route)}")
                 print(f"  Distance: {route_distance:.2f} km (Max: {model.R[k]:.2f})")
                 print(f"  Demand: {route_demand:.2f} kg (Capacity: {model.Q[k]:.2f})")
                 total_distance += route_distance
                 total_demand_served += route_demand
                 vehicles_used.add(k)
                 # Validation checks
                 if route_distance > model.R[k] + 1e-6: print(f"  WARNING: Vehicle {k} exceeded range!")
                 if route_demand > model.Q[k] + 1e-6: print(f"  WARNING: Vehicle {k} exceeded capacity!")
            elif len(route) > 1 : # Print incomplete/error routes if they exist
                 print(f"  Route Issue: {' -> '.join(route)}")
            # Else: already handled cases where vehicle didn't leave depot


    print("\n--- Overall Summary ---")
    print(f"Total distance covered by all vehicles: {total_distance:.2f} km")
    print(f"Total demand served: {total_demand_served:.2f} kg")
    print(f"Total expected demand: {sum(param_D_data.values()):.2f} kg")
    print(f"Number of vehicles used: {len(vehicles_used)} out of {len(model.K)}")
    if abs(total_demand_served - sum(param_D_data.values())) > 1e-6:
        print("WARNING: Total demand served does not match total expected demand!")


elif results.solver.termination_condition == pyo.TerminationCondition.infeasible:
     print("\n--- Model is Infeasible ---")
     print("The solver determined that there is no solution that satisfies all constraints.")
     print("Check constraints (capacities, ranges) and data for potential issues.")
     # Consider using Pyomo's tools for analyzing infeasibility if needed:
     # from pyomo.util.infeasible import log_infeasible_constraints
     # log_infeasible_constraints(model)

else:
    print("\n--- Solver did not find an Optimal or Feasible Solution ---")
    print(f"Solver Status: {results.solver.status}")
    print(f"Termination Condition: {results.solver.termination_condition}")
    print("Check solver logs and model formulation for errors.")
# --- End Python Code ---


Solving the model using gurobi...
Set parameter Username
Set parameter LicenseID to value 2656939
Academic license - for non-commercial use only - expires 2026-04-25
Read LP format model from file C:\Users\nicol\AppData\Local\Temp\tmpi2yt1i1d.pyomo.lp
Reading time = 0.03 seconds
x2773: 784 rows, 2773 columns, 11359 nonzeros
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 5 3500U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 784 rows, 2773 columns and 11359 nonzeros
Model fingerprint: 0x79c32267
Variable types: 55 continuous, 2718 integer (2718 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [2e+04, 8e+05]
  Bounds range     [1e+00, 9e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 3864126.9600
Presolve removed 434 rows and 1817 columns
Presolve time: 0.17

## 5. Generación de Archivos y Reportes

In [12]:
# --- Markdown ---
# ## 5. Generación de Archivo de Reporte Detallado por Vehículo
#
# Generate a single detailed CSV report summarizing each vehicle's route,
# including sequence, clients, demands, distance, and cost, matching the specific format requested.
# --- End Markdown ---

# --- Python Code ---
import os
import csv
import pyomo.environ as pyo # Asegúrate de que pyo esté disponible/importado

# Constante de costo por KM (debe coincidir con la usada en el modelo)
# Si la definiste en la sección 1, podrías pasarla como argumento,
# pero por simplicidad la redefinimos aquí si es necesario.
C_KM = 20700 # COP/km - ASEGÚRATE QUE ESTE VALOR SEA CORRECTO

# Función para generar el reporte CSV detallado - COMPLETAMENTE REESCRITA
def generate_detailed_vehicle_report(solution_model):
    """
    Genera un único archivo CSV detallado por vehículo con el formato especificado.
    Reconstruye las rutas y calcula las métricas directamente desde el modelo Pyomo solucionado.

    Args:
        solution_model (pyo.ConcreteModel): La instancia del modelo Pyomo resuelto.
                                            Se asume que contiene una solución óptima/factible.
    """
    # Obtener identificadores de variables de entorno o usar valores por defecto
    # AJUSTA ESTOS VALORES SI ES NECESARIO para que coincidan con tu caso específico
    group_name = os.getenv('GROUP_NAME', 'Grupo10')      # Nombre de tu grupo
    case_type = os.getenv('CASE_TYPE', 'estandar')     # Tipo de caso (estandar, sensible, etc.)
    case_number = os.getenv('CASE_NUMBER', '2')        # !!! IMPORTANTE: Usa el número correcto para 'case_2_base'

    print(f"\nGenerating DETAILED vehicle report for: {group_name}-caso-{case_type}-{case_number}")

    # Definir nombre del archivo de salida CSV detallado
    try:
        # Crear directorio si no existe
        output_dir = f"{group_name}-caso-{case_type}-{case_number}-results"
        os.makedirs(output_dir, exist_ok=True)
        report_filename = os.path.join(output_dir, f"{group_name}-caso-{case_type}-{case_number}-ReporteDetallado.csv")
        print(f"  Output directory: {output_dir}")
    except Exception as e:
        print(f"ERROR creating output directory or filename: {e}", file=sys.stderr)
        print("       Report will be saved in the current directory.")
        report_filename = f"{group_name}-caso-{case_type}-{case_number}-ReporteDetallado.csv"

    # --- Recopilación de Datos para el Reporte ---
    report_data_list = [] # Lista para guardar los diccionarios de cada fila del CSV
    assignments = {}      # Diccionario para guardar la asignación {vehículo: depot}

    print("  Reconstructing assignments and routes for detailed report...")
    # Paso 1: Determinar asignación vehículo -> depósito
    for k in solution_model.K:
        assignments[k] = None
        for i in solution_model.I:
            y_val = pyo.value(solution_model.y[i, k], exception=False)
            if y_val is not None and y_val > 0.5:
                assignments[k] = i
                break

    # Paso 2: Trazar ruta y calcular métricas para cada vehículo asignado y usado
    vehicles_reported_count = 0
    for k in solution_model.K:
        start_depot = assignments.get(k)
        if not start_depot: # Saltar si el vehículo no está asignado
            continue

        # Verificar si el vehículo realmente sale del depósito
        departs = any(pyo.value(solution_model.x[start_depot, v, k], exception=False) > 0.5
                      for v in solution_model.N if v != start_depot)
        if not departs: # Saltar si el vehículo está asignado pero no se usa
            # print(f"    Vehicle {k} assigned to {start_depot} but not used. Skipping report row.") # Debug
            continue

        # --- Inicio del Trazado y Cálculo de Métricas para Vehículo k ---
        vehicles_reported_count += 1
        route_nodes = [start_depot]            # Lista ordenada de nodos visitados
        visited_clients_ordered = []       # Lista ordenada de clientes visitados
        demands_ordered = []               # Lista ordenada de demandas satisfechas
        route_distance = 0.0               # Distancia total de la ruta
        current_node = start_depot
        route_valid = True                 # Flag para marcar si la ruta se trazó sin errores graves
        max_steps = len(solution_model.N) + 2

        # print(f"    Processing route for Vehicle {k} from {start_depot}...") # Debug

        for step in range(max_steps):
            next_node = None
            found_next_arc = False
            possible_next_nodes = []

            # Buscar el siguiente nodo
            for v in solution_model.N:
                if current_node != v:
                    x_val = pyo.value(solution_model.x[current_node, v, k], exception=False)
                    if x_val is not None and x_val > 0.5:
                        possible_next_nodes.append(v)
                        found_next_arc = True

            # Manejar ambigüedad o errores
            if len(possible_next_nodes) > 1:
                 print(f"    WARNING (Report Gen): Ambiguous path from {current_node} for {k}. Multiple next nodes: {possible_next_nodes}. Taking first: {possible_next_nodes[0]}.")
                 next_node = possible_next_nodes[0]
            elif len(possible_next_nodes) == 1:
                 next_node = possible_next_nodes[0]

            if not found_next_arc:
                if current_node == start_depot and step > 0: # Volvió al depósito
                    break
                else:
                    print(f"    ERROR (Report Gen): Route tracing for {k} stopped unexpectedly at {current_node}. Report row might be incomplete.")
                    route_valid = False
                    break

            # Obtener distancia del arco
            arc_dist = solution_model.d[current_node, next_node]
            if arc_dist == float('inf'):
                print(f"    ERROR (Report Gen): Route for {k} uses infeasible arc {current_node} -> {next_node}. Stopping trace.")
                route_valid = False
                break
            route_distance += arc_dist

            # Añadir nodo a la secuencia
            route_nodes.append(next_node)

            # Si es un cliente, registrarlo
            if next_node in solution_model.J:
                visited_clients_ordered.append(next_node)
                demands_ordered.append(solution_model.D[next_node])

            # Moverse al siguiente nodo
            current_node = next_node

            # Salir si se completó el ciclo
            if current_node == start_depot:
                break

            # Salir por seguridad (ciclo o pasos máx)
            if step == max_steps - 1:
                print(f"    WARNING (Report Gen): Route tracing for {k} exceeded max steps. Report row might be incomplete.")
                route_valid = False
                break
        # --- Fin del Trazado y Cálculo de Métricas ---

        if not route_valid:
            print(f"    Skipping report row generation for vehicle {k} due to route tracing errors.")
            continue # No añadir fila si hubo problemas graves en el trazado

        # --- Preparar Datos para la Fila del CSV ---
        # VehicleId: k (directamente)
        # DepotId: start_depot (directamente)
        # InitialLoad: Asumimos que es la capacidad del vehículo
        initial_load = solution_model.Q[k] # Usar .get por si acaso

        # RouteSequence: Unir la lista de nodos con ' - '
        route_sequence_str = ' - '.join(map(str, route_nodes))

        # ClientsServed: Contar clientes únicos visitados
        client_count = len(set(visited_clients_ordered)) # set() maneja visitas repetidas si las hubiera

        # DemandsSatisfied: Unir la lista de demandas con ' - '
        # Convertir demandas a string (y quizás a entero si son flotantes con .0)
        demands_str = ' - '.join(map(lambda x: str(int(x)) if x == int(x) else str(x), demands_ordered))

        # TotalDistance: route_distance (redondear opcionalmente)
        total_distance_rounded = round(route_distance, 2) # Redondear a 2 decimales

        # TotalTime: NO DISPONIBLE en el modelo actual. Usar placeholder.
        total_time = 0.0 # O podrías usar 'N/A' como string

        # FuelCost: Calcular basado en distancia y C_KM
        fuel_cost = round(route_distance * C_KM) # Redondear a entero

        # Crear el diccionario para esta fila
        row_data = {
            'VehicleId': k,
            'DepotId': start_depot,
            'InitialLoad': initial_load,
            'RouteSequence': route_sequence_str,
            'ClientsServed': client_count,
            'DemandsSatisfied': demands_str,
            'TotalDistance': total_distance_rounded,
            'TotalTime': total_time, # Placeholder
            'FuelCost': fuel_cost
        }
        report_data_list.append(row_data)
        # print(f"    Report row data prepared for {k}") # Debug

    print(f"  Collected data for {vehicles_reported_count} vehicles.")

    # --- Escribir el Archivo CSV Detallado ---
    if report_data_list: # Solo escribir si hay datos
        try:
            # Definir los encabezados EXACTAMENTE como los quieres
            headers = [
                'VehicleId', 'DepotId', 'InitialLoad', 'RouteSequence',
                'ClientsServed', 'DemandsSatisfied', 'TotalDistance',
                'TotalTime', 'FuelCost'
            ]
            with open(report_filename, mode='w', newline='', encoding='utf-8') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=headers)
                writer.writeheader()
                writer.writerows(report_data_list)
            print(f"- Detailed vehicle report created successfully: {report_filename}")
        except IOError as e:
            print(f"ERROR writing detailed report file {report_filename}: {e}", file=sys.stderr)
        except Exception as e:
             print(f"Unexpected ERROR writing detailed report file {report_filename}: {e}", file=sys.stderr)
    else:
        print(f"- No valid vehicle routes found to include in the detailed report: {report_filename}. File not written or empty.")

    # --- Opcional: Generar los otros archivos si aún los necesitas ---
    # (Podrías llamar aquí a la función anterior que generaba los otros 2 archivos
    # si quieres mantenerlos, pero esta función ahora se enfoca en el CSV detallado)
    # generate_objective_and_cost_files(solution_model) # Función hipotética

    print(f"Finished generating detailed report in directory: {os.path.abspath(output_dir)}")


# --- Llamada a la Generación de Reportes ---
# IMPORTANTE: Asume que las variables 'results' y 'model' están disponibles
#             después de ejecutar la Sección 4.
# Verifica la condición de terminación del solver *antes* de llamar a la función.

print("\nChecking solver status before attempting DETAILED report generation...")
if 'results' in locals() and 'model' in locals():
    term_cond = results.solver.termination_condition
    sol_status_ok = False
    if len(results.solution) > 0:
         sol_status_ok = True # Asumir OK si hay solución cargada

    if term_cond == pyo.TerminationCondition.optimal or \
       (term_cond == pyo.TerminationCondition.feasible and sol_status_ok) or \
       (term_cond == pyo.TerminationCondition.maxTimeLimit and sol_status_ok):

        print(f"  Solver status ({term_cond}) indicates a usable solution was found.")
        try:
             # Llamar a la función que genera el CSV detallado
             generate_detailed_vehicle_report(model)
        except Exception as e:
             print(f"\nERROR occurred during detailed report generation function call: {e}", file=sys.stderr)
             import traceback
             print("Traceback:")
             traceback.print_exc()

    else:
        print(f"  Solver termination condition ({term_cond}) does not indicate a usable solution.")
        if not sol_status_ok and len(results.solution) == 0:
             print("    Additionally, no solution data was loaded into the results object.")
        print("  Detailed report will not be generated.")

else:
    print("ERROR: Could not find 'results' or 'model' object from Section 4.", file=sys.stderr)
    print("       Cannot proceed with detailed report generation.", file=sys.stderr)
    print("       Please ensure Section 4 (Solver) has been executed successfully before this section.")

print("\n--- Section 5 Execution Finished ---")
# --- End Python Code ---


Checking solver status before attempting DETAILED report generation...
  Solver status (optimal) indicates a usable solution was found.

Generating DETAILED vehicle report for: Grupo10-caso-estandar-2
  Output directory: Grupo10-caso-estandar-2-results
  Reconstructing assignments and routes for detailed report...
  Collected data for 3 vehicles.
- Detailed vehicle report created successfully: Grupo10-caso-estandar-2-results\Grupo10-caso-estandar-2-ReporteDetallado.csv
Finished generating detailed report in directory: c:\Users\nicol\Desktop\CodingProjects\ISIS3302-ProyectoMOS\Grupo10-caso-estandar-2-results

--- Section 5 Execution Finished ---


## 6. Visualización con Mapas

In [13]:
# --- Markdown ---
# ## 6. Visualización con Mapas (Rutas por Carretera)
# --- End Markdown ---

# --- Python Code ---
import os
import pyomo.environ as pyo
import requests # Para llamadas API OSRM
import time     # Para delays entre llamadas API
import json     # Para parsear respuestas JSON

# Intenta importar librerías de visualización
try:
    import folium
    import matplotlib.pyplot as plt
    import matplotlib.colors as mcolors
    import numpy as np
    from branca.element import Template, MacroElement
    folium_installed = True
    print("Bibliotecas de visualización (folium, matplotlib) cargadas.")
except ImportError:
    print("ADVERTENCIA: Folium y/o Matplotlib no están instalados.")
    print("Para visualizar el mapa, por favor instálalos:")
    print("  pip install folium matplotlib numpy requests") # Añadido requests si no está
    folium_installed = False

# --- Definir URL del Servicio de RUTA OSRM ---
OSRM_ROUTE_URL = "http://router.project-osrm.org/route/v1/driving/" # URL para obtener geometría

# --- Verificar si se puede proceder con la visualización ---
can_visualize = False
if folium_installed and 'results' in locals() and 'model' in locals():
    # ... (la misma lógica de verificación de 'results' que antes) ...
    term_cond = results.solver.termination_condition
    sol_status_ok = (len(results.solution) > 0)
    if term_cond == pyo.TerminationCondition.optimal or \
       (term_cond == pyo.TerminationCondition.feasible and sol_status_ok) or \
       (term_cond == pyo.TerminationCondition.maxTimeLimit and sol_status_ok):
        print("Se encontró una solución válida. Procediendo a generar el mapa con rutas por carretera...")
        can_visualize = True
    else:
        print(f"\nEl estado del solver ({term_cond}) no indica una solución utilizable.")
        print("No se generará el mapa.")
# ... (resto de los mensajes de error si folium o results/model faltan) ...


# --- Funciones de Visualización (solo se definen si se pueden usar) ---
if can_visualize:

    # Función extract_routes_and_assignments_for_map (SIN CAMBIOS, la copiamos de la versión anterior)
    def extract_routes_and_assignments_for_map(solution_model):
        """Extrae rutas y asignaciones del modelo Pyomo resuelto."""
        routes = {}
        assignments = {}
        for k in solution_model.K:
            assignments[k] = None
            for i in solution_model.I:
                y_val = pyo.value(solution_model.y[i, k], exception=False)
                if y_val is not None and y_val > 0.5:
                    assignments[k] = i; break
        for v_id in solution_model.K:
            start_depot = assignments.get(v_id)
            routes[v_id] = []
            if not start_depot: continue
            departs = any(pyo.value(solution_model.x[start_depot, node, v_id], exception=False) > 0.5
                          for node in solution_model.N if node != start_depot)
            if not departs: continue
            route = [start_depot]; current_node = start_depot
            max_steps = len(solution_model.N) + 2; route_complete = False; error_in_route = False
            for step in range(max_steps):
                next_node = None; found_next = False; possible_next_nodes = []
                for j in solution_model.N:
                    if current_node != j:
                        x_val = pyo.value(solution_model.x[current_node, j, v_id], exception=False)
                        if x_val is not None and x_val > 0.5: possible_next_nodes.append(j); found_next = True
                if len(possible_next_nodes) > 1: next_node = possible_next_nodes[0] # Simplificación para el ejemplo
                elif len(possible_next_nodes) == 1: next_node = possible_next_nodes[0]
                if not found_next:
                    if current_node == start_depot and step > 0: route_complete = True
                    else: error_in_route = True
                    break
                route.append(next_node); current_node = next_node
                if current_node == start_depot: route_complete = True; break
                if step == max_steps - 1: error_in_route = True; break
            if route_complete and not error_in_route: routes[v_id] = route
            elif route: routes[v_id] = [] # Marcar como vacía si hay error
        return routes, assignments

    # Función create_map_base (SIN CAMBIOS)
    def create_map_base(data_dict):
        coords_dict = data_dict.get('UbicacionNodo'); default_location = [4.6097, -74.0817]; zoom_start = 11
        if not coords_dict: return folium.Map(location=default_location, zoom_start=zoom_start, tiles='CartoDB positron')
        lats = [loc[0] for loc in coords_dict.values() if isinstance(loc, (list, tuple)) and len(loc) == 2]
        longs = [loc[1] for loc in coords_dict.values() if isinstance(loc, (list, tuple)) and len(loc) == 2]
        if not lats or not longs: return folium.Map(location=default_location, zoom_start=zoom_start, tiles='CartoDB positron')
        avg_lat = sum(lats) / len(lats); avg_long = sum(longs) / len(longs)
        lat_span = max(lats) - min(lats); lon_span = max(longs) - min(longs)
        if lat_span < 0.1 and lon_span < 0.1: zoom_start = 14
        elif lat_span < 0.5 and lon_span < 0.5: zoom_start = 12
        else: zoom_start = 11
        return folium.Map(location=[avg_lat, avg_long], zoom_start=zoom_start, tiles='CartoDB positron')


    # Función add_map_legend (SIN CAMBIOS)
    def add_map_legend(m, vehicle_colors):
        items = ''.join(f'<div style="margin-bottom: 5px;"><i style="background:{color}; width:15px; height:15px; float:left; margin-right:8px; border:1px solid grey; border-radius: 50%;"></i>{vehicle}</div>'
                        for vehicle, color in vehicle_colors.items() if color)
        if not items: return
        legend_html = f'''<div style="position: fixed; bottom: 30px; left: 10px; width: auto; max-width: 180px; height: auto; z-index:9999; font-size:12px; background-color: rgba(255, 255, 255, 0.85); padding: 10px; border:1px solid grey; border-radius:8px; box-shadow: 3px 3px 5px rgba(0,0,0,0.3);"><h4 style="margin-top:0; margin-bottom:10px; font-weight:bold; text-align:center;">Vehículos</h4>{items}</div>'''
        legend = MacroElement(); legend._template = Template(legend_html); m.get_root().add_child(legend)


    # Función para plotear rutas y marcadores (MODIFICADA PARA USAR OSRM)
    def plot_routes_and_markers(m, routes, assignments, data_dict, solution_model):
        """Dibuja las rutas (usando OSRM para geometría) y los marcadores."""
        coords_dict = data_dict.get('UbicacionNodo', {})
        if not coords_dict:
            print("Error Mapeo: No hay coordenadas ('UbicacionNodo') para plotear.")
            return

        active_routes = {v: r for v, r in routes.items() if r and len(r) > 1}
        num_active_vehicles = len(active_routes)
        vehicle_colors = {}
        if num_active_vehicles > 0:
            cmap = plt.cm.get_cmap('tab10', num_active_vehicles)
            active_vehicle_ids = sorted(active_routes.keys())
            for idx, v_id in enumerate(active_vehicle_ids):
                 vehicle_colors[v_id] = mcolors.rgb2hex(cmap(idx % cmap.N))
        else:
             print("Advertencia Mapeo: No hay rutas activas para visualizar.")

        # --- 1. Dibujar las RUTAS usando OSRM ---
        print(f"Obteniendo geometría de rutas desde OSRM para {num_active_vehicles} vehículos activos...")
        for i, (v_id, route_nodes) in enumerate(active_routes.items()):
            color = vehicle_colors.get(v_id, '#808080')
            print(f"  Procesando ruta para Vehículo {v_id} ({i+1}/{num_active_vehicles})...")

            # Obtener coordenadas y formatear para OSRM (lon,lat;lon,lat...)
            osrm_coords_str = ""
            route_coords_list = [] # Lista de (lat, lon) para fallback
            valid_coords = True
            for node_id in route_nodes:
                coords = coords_dict.get(node_id)
                if coords and isinstance(coords, (list, tuple)) and len(coords) == 2:
                    lat, lon = coords
                    route_coords_list.append((lat, lon))
                    osrm_coords_str += f"{lon},{lat};" # Formato OSRM: lon,lat
                else:
                    print(f"    ERROR: Coordenada inválida/faltante para nodo {node_id} en ruta {v_id}. No se puede consultar OSRM.")
                    valid_coords = False
                    break
            if not valid_coords or len(route_coords_list) < 2:
                print(f"    Saltando consulta OSRM para ruta de {v_id}.")
                continue

            osrm_coords_str = osrm_coords_str.rstrip(';') # Quitar último ';'

            # Construir URL y parámetros para OSRM
            osrm_request_url = f"{OSRM_ROUTE_URL}{osrm_coords_str}"
            params = {
                'overview': 'full',
                'geometries': 'geojson',
                'steps': 'false',
                'alternatives': 'false',
                # 'annotations': 'false' # Opcional: reducir tamaño respuesta
            }

            # --- Llamada a OSRM con manejo de errores y delay ---
            route_geojson = None
            try:
                # Introducir un pequeño delay para ser amable con el servidor público
                time.sleep(0.6) # ~1-2 llamadas por segundo máx

                response = requests.get(osrm_request_url, params=params, timeout=15) # Timeout de 15s
                response.raise_for_status() # Lanza error para respuestas 4xx/5xx
                route_data = response.json()

                if route_data.get('code') == 'Ok' and route_data.get('routes'):
                    # Extraer la geometría GeoJSON
                    route_geojson = route_data['routes'][0]['geometry']
                    print(f"      Geometría OSRM obtenida.")
                else:
                    print(f"    ADVERTENCIA: Respuesta OSRM no fue 'Ok' o no contenía rutas para {v_id}. Código: {route_data.get('code')}")

            except requests.exceptions.RequestException as e:
                print(f"    ERROR: Falló la consulta a OSRM para la ruta de {v_id}: {e}")
            except json.JSONDecodeError:
                print(f"    ERROR: No se pudo decodificar la respuesta JSON de OSRM para {v_id}.")
            except Exception as e: # Capturar otros errores inesperados
                print(f"    ERROR inesperado durante consulta OSRM para {v_id}: {e}")

            # --- Dibujar en el mapa ---
            if route_geojson:
                # Dibujar usando GeoJson si OSRM funcionó
                style_function = lambda x, color=color: {
                    'color': color,
                    'weight': 3.5,
                    'opacity': 0.85
                }
                folium.GeoJson(
                    route_geojson,
                    name=f'Ruta {v_id}',
                    style_function=style_function,
                    tooltip=f"Ruta Vehículo {v_id} (OSRM)"
                ).add_to(m)
            else:
                # Fallback: Dibujar línea recta si OSRM falló
                print(f"    FALLBACK: Dibujando línea recta para la ruta de {v_id}.")
                folium.PolyLine(
                    locations=route_coords_list, # Usar la lista (lat, lon)
                    color=color,
                    weight=2.5, # Más delgada para indicar fallback
                    opacity=0.6,
                    dash_array='5, 5', # Línea punteada para indicar fallback
                    tooltip=f"Ruta Vehículo {v_id} (Recta - Falló OSRM)"
                ).add_to(m)

        # --- 2. Añadir MARCADORES para todos los nodos (Lógica sin cambios) ---
        print("\nAñadiendo marcadores de nodos...")
        all_node_ids = set(solution_model.N)
        for node_id in all_node_ids:
            coords = coords_dict.get(node_id)
            if not coords or not isinstance(coords, (list, tuple)) or len(coords) != 2: continue
            lat, lon = coords
            node_type = 'Cliente' if node_id in solution_model.J else 'Depósito' if node_id in solution_model.I else 'Desconocido'
            demand = 0; marker_color = 'gray'; icon_name = 'question'; vehicle_info = "No Asignado/Visitado"
            popup_text = f"<b>Nodo:</b> {node_id}<br><b>Tipo:</b> {node_type}"

            if node_type == 'Depósito':
                icon_name = 'industry'; vehicles_at_depot = [k for k, dep in assignments.items() if dep == node_id]
                if vehicles_at_depot:
                    marker_color = 'darkblue'; active_vehicles_str = ", ".join([f"{k}{'*' if k in active_routes else ''}" for k in vehicles_at_depot])
                    vehicle_info = f"Base de: {active_vehicles_str} (*=activa)"
                else: marker_color = 'lightblue'; vehicle_info = "No utilizado"
                popup_text += f"<br>{vehicle_info}"
            elif node_type == 'Cliente':
                icon_name = 'user'; vehicle_visiting = None
                try: demand = solution_model.D[node_id].value
                except: pass # Ignorar error de demanda si no existe
                for v_id_active in active_routes.keys():
                    is_visited = any((pyo.value(solution_model.x[u, node_id, v_id_active], exception=False) or 0) > 0.5 for u in solution_model.N if u != node_id)
                    if is_visited: vehicle_visiting = v_id_active; break
                if vehicle_visiting: marker_color = vehicle_colors.get(vehicle_visiting, 'green'); vehicle_info = f"Servido por {vehicle_visiting}"
                else: marker_color = 'red'; vehicle_info = "NO SERVIDO" # Simplificado
                popup_text += f"<br><b>Demanda:</b> {demand:.2f}"
                popup_text += f"<br>{vehicle_info}"
            else: marker_color = 'black'; icon_name = 'times'

            icon = folium.Icon(color=marker_color, icon=icon_name, prefix='fa')
            folium.Marker(location=[lat, lon], icon=icon, popup=folium.Popup(popup_text, max_width=250, sticky=True), tooltip=f"Nodo {node_id} ({node_type})").add_to(m)

        # --- 3. Añadir la leyenda ---
        add_map_legend(m, vehicle_colors)


    # --- Generación y Visualización del Mapa ---
    print("Generando mapa con rutas por carretera...")

    # ... (obtener group_name, case_type, case_number como antes) ...
    group_name = os.getenv('GROUP_NAME', 'Grupo10')
    case_type = os.getenv('CASE_TYPE', 'estandar')
    case_number = os.getenv('CASE_NUMBER', '2') # Asegúrate que coincida

    # 1. Extraer rutas y asignaciones
    routes_map, assignments_map = extract_routes_and_assignments_for_map(model)

    # 2. Crear mapa base
    map_viz = create_map_base(data)

    # 3. Plotear rutas (con OSRM) y marcadores
    plot_routes_and_markers(map_viz, routes_map, assignments_map, data, model)

    # 4. Guardar y mostrar
    try:
        output_dir = f"{group_name}-caso-{case_type}-{case_number}-results"
        os.makedirs(output_dir, exist_ok=True)
        map_filename = os.path.join(output_dir, f"{group_name}-caso-{case_type}-{case_number}-mapa_rutas_carretera.html") # Nuevo nombre
        map_viz.save(map_filename)
        print(f"\nMapa guardado exitosamente en: '{os.path.abspath(map_filename)}'")
        try:
            from IPython.display import display
            print("Intentando mostrar el mapa...")
            display(map_viz)
        except ImportError: print("IPython no disponible. Abre el archivo HTML manualmente.")
    except Exception as e:
        print(f"\nERROR al guardar o mostrar el mapa: {e}")
        import traceback; print("Traceback:"); traceback.print_exc()

# --- Mensaje final si no se pudo visualizar ---
# (Se imprime al principio)

print("\n--- Section 6 Execution Finished ---")
# --- End Python Code ---

Bibliotecas de visualización (folium, matplotlib) cargadas.
Se encontró una solución válida. Procediendo a generar el mapa con rutas por carretera...
Generando mapa con rutas por carretera...
Obteniendo geometría de rutas desde OSRM para 3 vehículos activos...
  Procesando ruta para Vehículo V1 (1/3)...


C:\Users\nicol\AppData\Local\Temp\ipykernel_19900\462578454.py:125: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = plt.cm.get_cmap('tab10', num_active_vehicles)


      Geometría OSRM obtenida.
  Procesando ruta para Vehículo V2 (2/3)...
      Geometría OSRM obtenida.
  Procesando ruta para Vehículo V3 (3/3)...
      Geometría OSRM obtenida.

Añadiendo marcadores de nodos...


C:\Users\nicol\AppData\Local\Temp\ipykernel_19900\462578454.py:249: UserWarning: color argument of Icon should be one of: {'lightred', 'pink', 'blue', 'beige', 'lightgray', 'darkpurple', 'orange', 'darkblue', 'cadetblue', 'darkgreen', 'lightgreen', 'darkred', 'purple', 'black', 'white', 'green', 'red', 'lightblue', 'gray'}.
  icon = folium.Icon(color=marker_color, icon=icon_name, prefix='fa')



Mapa guardado exitosamente en: 'c:\Users\nicol\Desktop\CodingProjects\ISIS3302-ProyectoMOS\Grupo10-caso-estandar-2-results\Grupo10-caso-estandar-2-mapa_rutas_carretera.html'
Intentando mostrar el mapa...



--- Section 6 Execution Finished ---
